# Stage 3: 1cSVM for Anomaly Detection for IoT [1,3,5,6]
The 1cSVM  anomaly detection model used in this project work as follow:
## 1. We  compressed the data using  AE model. The compressed  data from AE output  from that  was used as input data for 1cSVM.
## 2. The data was split into train and test and put through 1cSVM and a confusion matrix was drawn.


In [1]:
from mmae.multimodal_autoencoder import MultimodalAutoencoder

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping
from keras import backend, Model, regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

from scipy import spatial

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import seed
seed(42)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

Using TensorFlow backend.


### Device 1,3,5,6 input data as pkl format

In [2]:
# Import and stitch all benign data together

device1 = pd.read_csv("1.benign.csv")
device3 = pd.read_csv("3.benign.csv")
device5 = pd.read_csv("5.benign.csv")
device6 = pd.read_csv("6.benign.csv")

all_benign = pd.concat([device1,device3,device5,device6])
all_benign['label'] = 0

cut_off = int(all_benign.shape[0]/2)

# Split the benign data into train and test
all_benign, all_benign_test,_,_ = train_test_split(all_benign.iloc[:,:-1].values, 
                                all_benign.iloc[:,-1].values, test_size=0.5, random_state=42)

# Scale the train data
scaler = StandardScaler()
scaler.fit(all_benign)
all_benign = scaler.transform(all_benign)

print(all_benign.shape)

(124658, 115)


In [3]:

# function to calculate rmse
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# Auto Encoder inout layer
input_layer = Input(shape=(all_benign.shape[1],))

# Encoder part
encoded = Dense(110, activation='relu', activity_regularizer=regularizers.l1_l2(l1=10e-6, l2=10e-6))(input_layer)
encoded = Dense(95, activation='relu')(encoded)
encoded = Dense(20)(encoded)

# Decoder part
decoded = Dense(95, activation='relu')(encoded)
decoded = Dense(110, activation='relu')(decoded)

# Output layer
output_layer = Dense(all_benign.shape[1])(decoded)

# Compile autoencoder
autoencoder = Model(input_layer, output_layer)
encoder = Model(input_layer, encoded)
autoencoder.compile(optimizer= 'adadelta', loss=rmse)

# fit the model
autoencoder.fit(all_benign, all_benign, batch_size = 100, epochs = 450,
                validation_split = 0.20, callbacks=[EarlyStopping(patience=50,monitor='val_loss')])

# predict on validation and other benign 
benign1_test = encoder.predict(all_benign)

Train on 99726 samples, validate on 24932 samples
Epoch 1/450
99726/99726 [==============================] - 5s 47us/step - loss: 0.2810 - val_loss: 0.1960
Epoch 2/450
99726/99726 [==============================] - 4s 36us/step - loss: 0.1637 - val_loss: 0.1369
Epoch 3/450
99726/99726 [==============================] - 4s 36us/step - loss: 0.1323 - val_loss: 0.1643
Epoch 4/450
99726/99726 [==============================] - 4s 35us/step - loss: 0.1193 - val_loss: 0.1228
Epoch 5/450
99726/99726 [==============================] - 4s 43us/step - loss: 0.1118 - val_loss: 0.1265
Epoch 6/450
99726/99726 [==============================] - 4s 37us/step - loss: 0.1058 - val_loss: 0.1157
Epoch 7/450
99726/99726 [==============================] - 4s 39us/step - loss: 0.1014 - val_loss: 0.1042
Epoch 8/450
99726/99726 [==============================] - 3s 32us/step - loss: 0.0982 - val_loss: 0.1073
Epoch 9/450
99726/99726 [==============================] - 5s 49us/step - loss: 0.0943 - val_loss: 0.0

99726/99726 [==============================] - 4s 35us/step - loss: 0.0489 - val_loss: 0.0537
Epoch 154/450
99726/99726 [==============================] - 3s 34us/step - loss: 0.0487 - val_loss: 0.0511
Epoch 155/450
99726/99726 [==============================] - 4s 38us/step - loss: 0.0488 - val_loss: 0.0558
Epoch 156/450
99726/99726 [==============================] - 4s 41us/step - loss: 0.0486 - val_loss: 0.0571
Epoch 157/450
99726/99726 [==============================] - 3s 34us/step - loss: 0.0486 - val_loss: 0.0597
Epoch 158/450
99726/99726 [==============================] - 4s 36us/step - loss: 0.0485 - val_loss: 0.0589
Epoch 159/450
99726/99726 [==============================] - 4s 36us/step - loss: 0.0485 - val_loss: 0.0531
Epoch 160/450
99726/99726 [==============================] - 4s 36us/step - loss: 0.0486 - val_loss: 0.0551
Epoch 161/450
99726/99726 [==============================] - 4s 39us/step - loss: 0.0484 - val_loss: 0.0519
Epoch 162/450
99726/99726 [===============

In [4]:
# Read and combine anomaly data
anomaly = pd.concat([pd.read_csv("1.gafgyt.combo.csv"), pd.read_csv("1.gafgyt.junk.csv"), pd.read_csv("1.gafgyt.scan.csv"),pd.read_csv("1.gafgyt.tcp.csv"), pd.read_csv("1.gafgyt.udp.csv"), pd.read_csv("1.mirai.ack.csv"),pd.read_csv("1.mirai.scan.csv"), pd.read_csv("1.mirai.syn.csv"), pd.read_csv("1.mirai.udp.csv"), pd.read_csv("1.mirai.udpplain.csv"),pd.read_csv("3.gafgyt.combo.csv"), pd.read_csv("3.gafgyt.junk.csv"), pd.read_csv("3.gafgyt.scan.csv"),pd.read_csv("3.gafgyt.tcp.csv"), pd.read_csv("3.gafgyt.udp.csv"),pd.read_csv("5.gafgyt.combo.csv"), pd.read_csv("5.gafgyt.junk.csv"), pd.read_csv("5.gafgyt.scan.csv"),pd.read_csv("5.gafgyt.tcp.csv"), pd.read_csv("5.gafgyt.udp.csv"), pd.read_csv("5.mirai.ack.csv"),pd.read_csv("5.mirai.scan.csv"), pd.read_csv("5.mirai.syn.csv"), pd.read_csv("5.mirai.udp.csv"), pd.read_csv("5.mirai.udpplain.csv"),pd.read_csv("6.gafgyt.combo.csv"), pd.read_csv("6.gafgyt.junk.csv"), pd.read_csv("6.gafgyt.scan.csv"),pd.read_csv("6.gafgyt.tcp.csv"), pd.read_csv("6.gafgyt.udp.csv"), pd.read_csv("6.mirai.ack.csv"),pd.read_csv("6.mirai.scan.csv"), pd.read_csv("6.mirai.syn.csv"), pd.read_csv("6.mirai.udp.csv"), pd.read_csv("6.mirai.udpplain.csv")])

column_names = [anomaly .columns.values]



In [5]:
# Scale and label anomaly data
anomaly = scaler.transform(anomaly)
anomaly = pd.DataFrame(anomaly,columns = column_names)
anomaly['label'] = 1

# Scale and label benign test data
all_benign_test = scaler.transform(all_benign_test)
all_benign_test = pd.DataFrame(all_benign_test, columns = column_names)
all_benign_test['label'] = 0

#get the intial training data as well
benign1_test = pd.DataFrame(benign1_test)
benign1_test['Label'] = 0

print(anomaly.shape)
print(all_benign_test.shape)
print(benign1_test.shape)

# Combine benign and anomaly data
test_data = pd.concat([anomaly,all_benign_test])

test_data = test_data.sample(frac=1, random_state=42)

compressed_data = encoder.predict(test_data.iloc[:,:-1])
compressed_data = pd.DataFrame(compressed_data)
compressed_data['Label'] = test_data.iloc[:,-1].values
compressed_data = pd.concat([compressed_data,benign1_test])

del all_benign_test, anomaly, benign1_test, test_data

print(compressed_data.head())
print(compressed_data.tail())
print(compressed_data.shape)



(2789633, 116)
(124658, 116)
(124658, 21)
            0          1           2          3           4           5  \
0 -136.372055 -36.018013  -61.650940 -36.265022   72.786034  -92.512077   
1   -4.896306   2.543107   -2.749959  -0.095127   -1.660665    1.964495   
2   -4.896306   2.543107   -2.749959  -0.095127   -1.660665    1.964495   
3 -329.659851 -64.248611 -154.619202  -5.011531  116.735481 -136.044571   
4 -111.940994 -21.782181  -52.776093  -1.488158   39.353992  -46.589722   

           6           7          8          9         10         11  \
0  -3.196047  -22.852793 -54.583939 -12.091518  40.412018  18.692213   
1  -0.977479    4.931548   0.877410  -2.465756  -1.068041  -0.364596   
2  -0.977479    4.931548   0.877410  -2.465756  -1.068041  -0.364596   
3 -93.104553 -105.933609 -74.801414 -46.731384  78.231842   8.056463   
4 -33.005077  -33.291275 -26.322390 -18.796240  24.386293  -0.903177   

          12        13          14          15          16         17  \
0

In [9]:
# seperate out benign and anomaly data
benign_1356 = compressed_data.loc[compressed_data['Label'] == 0]
anomaly_1356 = compressed_data.loc[compressed_data['Label'] == 1]

benign_1356.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,Label
30,-0.632992,0.116820,-0.541826,1.783621,0.282258,-0.311032,-0.569160,0.795657,0.354573,0.001910,0.714400,0.043639,0.710571,0.454172,1.088213,-0.754556,-0.454888,-0.654312,-1.989696,-0.789233,0
31,0.171656,-0.248414,-0.048502,0.160808,-0.135926,0.100528,0.449040,0.137520,-0.024721,-0.267727,0.050313,-0.039022,-0.173164,0.296946,0.257075,0.069791,0.107526,-0.369945,0.250403,-0.183586,0
59,-0.143467,1.054214,0.207745,2.439886,0.254439,0.540787,0.406138,0.198254,-0.300267,0.287549,0.527818,-0.441936,0.221852,0.224520,0.948814,0.059361,-0.113230,-0.037674,-1.600441,-0.237981,0
65,0.116786,-0.142173,0.138746,0.072872,-0.404291,0.192383,0.097246,-0.213578,0.037728,-0.315667,0.016362,0.089543,-0.014961,-0.045144,0.022388,0.123795,-0.064604,-0.164441,0.064711,0.005760,0
119,0.057020,0.753345,-0.201065,2.077771,0.286774,0.189470,-0.263567,1.043588,0.309579,0.324522,0.365582,-0.124891,0.057770,0.640336,1.635486,-0.202857,0.346579,-0.289535,-1.173367,-0.866696,0


In [11]:
# remove the label from benign data
benign_1356 = benign_1356.drop(['Label'], axis=1)
benign_1356.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
30,-0.632992,0.116820,-0.541826,1.783621,0.282258,-0.311032,-0.569160,0.795657,0.354573,0.001910,0.714400,0.043639,0.710571,0.454172,1.088213,-0.754556,-0.454888,-0.654312,-1.989696,-0.789233
31,0.171656,-0.248414,-0.048502,0.160808,-0.135926,0.100528,0.449040,0.137520,-0.024721,-0.267727,0.050313,-0.039022,-0.173164,0.296946,0.257075,0.069791,0.107526,-0.369945,0.250403,-0.183586
59,-0.143467,1.054214,0.207745,2.439886,0.254439,0.540787,0.406138,0.198254,-0.300267,0.287549,0.527818,-0.441936,0.221852,0.224520,0.948814,0.059361,-0.113230,-0.037674,-1.600441,-0.237981
65,0.116786,-0.142173,0.138746,0.072872,-0.404291,0.192383,0.097246,-0.213578,0.037728,-0.315667,0.016362,0.089543,-0.014961,-0.045144,0.022388,0.123795,-0.064604,-0.164441,0.064711,0.005760
119,0.057020,0.753345,-0.201065,2.077771,0.286774,0.189470,-0.263567,1.043588,0.309579,0.324522,0.365582,-0.124891,0.057770,0.640336,1.635486,-0.202857,0.346579,-0.289535,-1.173367,-0.866696


In [12]:
# train test split on benign data into 50% train and 50% test
all_benign_train1356, all_benign_test1356  = train_test_split(benign_1356, test_size=0.5, random_state=42)


In [13]:
# fit a one class SVM on benign train data
from sklearn.svm import OneClassSVM

oneclass1356 = OneClassSVM(kernel='rbf', gamma=.02,nu=0.02).fit(all_benign_train1356)



In [15]:
# create test data

# change the all_benign_test label to 1, create the label dataframe and drop the label from original dataframe
all_benign_test1356['Label'] = 0
benign_label1356 = all_benign_test1356['Label'] + 1
all_benign_test1356 = all_benign_test1356.drop(['Label'], axis=1)

# change the anomlay label to -1, create the label dataframe and drop the label from original dataframe
anomaly_label1356 = anomaly_1356['Label']-2
anomaly_1356 = anomaly_1356.drop(['Label'], axis=1)

# append benign and anonlay data to create overall test data and test label
test_data1356 = all_benign_test1356.append(anomaly_1356)
test_label1356 = benign_label1356.append(anomaly_label1356)

# rune the one class svm prediction on test data
prediction1356 = oneclass1356.predict(test_data1356)

In [16]:
# confusion matrix

cm = confusion_matrix(test_label1356 , prediction1356)
cm

array([[2789436,     197],
       [   2694,  121964]], dtype=int64)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(test_label1356 , prediction1356))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00   2789633
           1       1.00      0.98      0.99    124658

    accuracy                           1.00   2914291
   macro avg       1.00      0.99      0.99   2914291
weighted avg       1.00      1.00      1.00   2914291



In [18]:
#Confusion matrix - Percentage
cm = pd.DataFrame(confusion_matrix(test_label1356 , prediction1356,normalize = 'true',labels=[-1,1]),index=['0', '1'],columns = ['0', '1'])
cm

,0,1
0,0.999929,0.000071
1,0.021611,0.978389
